In [1]:
import pandas as pd
import numpy as np
import os
import geopandas as gd
import openpyxl as px
import psycopg2 as pg
from python_calamine.pandas import pandas_monkeypatch
import configparser
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
#Abrindo monkeypatch para abrir o calamine

pandas_monkeypatch()

In [3]:
#path premissas 
path_premissas = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\001. Rolling Forcast - RF\Distribuição Regional\premissas.xlsx'

In [4]:
crendenciais_path = r'F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\04. Pessoas\Gabriel Moura\00 - Silvicultura\00 - Credenciais\credenciais.conf'

In [5]:
#Senhas e credênciais 

credenciais = configparser.ConfigParser()

credenciais.read(crendenciais_path)

logins = credenciais['credenciais']


conn_dwh = pg.connect(
    user =logins['login'],
    password =logins['senha_sql'],
    host = logins['host'],
    port = logins['port'],
    database = logins['data_base']
)

In [6]:
#Mascara mês operacional

def mes_operacional(df, coluna, nome_coluna):
    # Garantir que a coluna é do tipo datetime
    if not pd.api.types.is_datetime64_any_dtype(df[coluna]):
        df[coluna] = pd.to_datetime(df[coluna], errors='coerce')
    
    
    def calcular_mes_operacional(data):
        
        if pd.isna(data):
            return None 
        
        
        dia = data.day
        mes = data.month
        ano = data.year
        
        
        if dia > 20:
            mes = (mes % 12) + 1
            if mes == 1:
                ano += 1
        
        # Retornar um Timestamp, garantindo que todos são inteiros
        return pd.Timestamp(year=int(ano), month=int(mes), day=1)

    # Aplicar a função para a coluna
    df[nome_coluna] = df[coluna].apply(calcular_mes_operacional)

    return df

In [7]:
#Mascara Talhão

def mascara(df, coluna,n_digitos):
    df[coluna] = df[coluna].astype(str)
    df[coluna] = str("000") + df[coluna]
    df[coluna] = df[coluna].astype(str).str[-n_digitos:] 
    return df

In [8]:
#Base de CTO de baldeio 

cursor_cto = conn_dwh.cursor()

cursor_cto.execute(

"""
select 
cd_uso_solo,
id_projeto,
cd_talhao,
dcr_produto,
dcr_operacao,
data_cto,
flag_cto_executado

from iforestry.vw_cubo_col_talhoes_cto 

where 

dcr_regiao = 'BRACELL SP' 

order by
	cd_uso_solo asc,
	data_cto DESC

"""

)

dados_cto_baldeio  = cursor_cto.fetchall()

cursor_cto.close()


colunas_cto = [desc[0] for desc in cursor_cto.description]

cto = pd.DataFrame(dados_cto_baldeio, columns=colunas_cto)

In [9]:
#Base CTO

#Convertendo para o formato de data
cto['data_cto'] = pd.to_datetime(cto['data_cto'],errors='coerce')

#Ordenando de forma a manter o apontamento mais rescente primeiro
cto = cto.sort_values(['cd_uso_solo','data_cto'], ascending=[False,False])

#Revomendo duplicatas para ter apenas o apontamento mais rescente
#cto = cto.drop_duplicates('cd_uso_solo',keep='first')


cto = mascara(cto,'id_projeto',4)
cto = mascara(cto,'cd_talhao',6)
cto['Objeto de locação'] = cto['id_projeto'] + cto['cd_talhao']



cto['cto baldeio'] = np.where( (cto['dcr_operacao'] == 'BALDEIO FORWARDER') | (cto['dcr_operacao'] == 'ARRASTE SKIDDER') , cto['data_cto'],None)

cto['cto colheita'] = np.where( (cto['dcr_operacao'] == 'CORTE HARVESTER') | (cto['dcr_operacao'] == 'CORTE FELLER'), cto['data_cto'],None)


cto['cto baldeio'] = pd.to_datetime(cto['cto baldeio'],errors='coerce')

cto['cto colheita'] = pd.to_datetime(cto['cto colheita'],errors='coerce')



cto = cto.sort_values('data_cto', ascending=False)

cto = cto[pd.notna(cto['data_cto'])]

#cto = cto.drop_duplicates('Objeto de locação', keep='first')


cto_agg = {'cto baldeio': 'max',
           'cto colheita' : 'max'}


#Agrupando para sempre ter duas datas em uma mesma linha
cto = cto.groupby(['cd_uso_solo', 'Objeto de locação']).agg(cto_agg).reset_index()

#cto.to_excel('cto.xlsx')


In [10]:
#def Premissas

premissas_df = pd.read_excel(path_premissas, sheet_name='path')

def input_file(file):
    premissas = premissas_df.copy()
    
    premissas = premissas[premissas['status'] == 'ativo']

    premissas = premissas[premissas['arquivos de consumo'] == file]

    path = premissas['path'].iloc[0]

    return path

In [11]:
#Função ajuste de objeto de locação

def ajuste_obj(df, colums):
    df[colums] = df[colums].apply(lambda x: f"{str(x).split('-')[0]}-01" if pd.notna(x) else x)


In [12]:
#Coppice

path_coppice = input_file('path_coppice')

#Lendo o excel para encontrar quais arquivos possuem
excel_file_coppice = pd.ExcelFile(path_coppice)

#List comprehension
sheet_names_coppice = [sheet for sheet in excel_file_coppice.sheet_names if sheet.startswith('RF')]


coppice = pd.read_excel(path_coppice, dtype={'Proj & Tal':'object'},
                         sheet_name=sheet_names_coppice[0], 
                         usecols=['Proj & Tal', 'Área(ha)','IMA current', 'Aprovação']).rename({'Proj & Tal':'Objeto de locação','Área(ha)':'ha coppice', 'IMA current':'IMA'}, 
                         axis=1)

coppice = coppice[pd.isna(coppice['Aprovação'])]

coppice = coppice.drop('Aprovação', axis=1)

coppice['coppice'] = 'coppice'

coppice = mascara(coppice,'Objeto de locação', 7)


#Ajustando talhão no coppice
ajuste_obj(df=coppice, colums='Objeto de locação')

In [13]:
#Rahab

path_rehab = input_file('path_rehab')

rehab = pd.read_excel(path_rehab, 
                      sheet_name='Geral', 
                      dtype={'Talhão':'object' },
                      usecols=['Talhão','Origem','Área','Sequência', 'Data Start Rehab']).rename({'Talhão':'Objeto de locação', 'Origem': 'Rehab', 'Área':'area rehab'}, axis=1)

#Filtrando o que é rehab
rehab = rehab[((rehab['Rehab'] == 'Rehab DO') & (pd.notna(rehab['Data Start Rehab']))) 
              | (rehab['Rehab'] == 'Rehab Silv')]

rehab['Data Start Rehab'] = pd.to_datetime(rehab['Data Start Rehab'], unit='D', origin='1899-12-30')



#Removendo duplicatas
rehab = rehab.drop_duplicates('Objeto de locação', keep='first')


#Ajustando talhão no rehab
ajuste_obj(df=rehab, colums='Objeto de locação')

rehab['Rehab status'] = 'Rehab'

In [14]:
#Espaçamento 3,8 x 2,4




path_espacamento = input_file('path_espacamento')


esp = pd.read_excel(path_espacamento,
                    sheet_name='BD',
                    dtype={'ID Projeto':'object',
                           'Talhão':'object'},
                    usecols=['ID Projeto','Talhão']       
                           )

esp['Espaçamento P&D'] = 'Espaçamento 3,8 x 2,4'

esp['Objeto de locação'] = esp['ID Projeto'].astype(str) + esp['Talhão'].astype(str)

esp = esp.drop(['ID Projeto', 'Talhão'], axis=1)

ajuste_obj(esp,'Objeto de locação')


In [15]:
#Cadastro - query


cursor_cadastro = conn_dwh.cursor()

cursor_cadastro.execute(
"""
	
SELECT
    id_carac_uso_solo_grp || '-' || dcr_carac_uso_solo_grp AS "Uso do Talhão",
    id_carac_uso_solo_detalhe || '-' || dcr_carac_uso_solo_detalhe as "Descrição do Uso do talhão",
    id_projeto || cd_talhao AS "Objeto de locação",
    cd_uso_solo,
    dcr_tipo_uso_solo AS "Tipo Propriedade" ,
    dcr_regiao AS "Região",
    id_projeto AS "Id Projeto",
    nom_projeto AS "Projeto",
    cd_talhao AS "Talhão",
    num_ciclo AS "Ciclo",
    num_rotacao AS "Rotação",
    dcr_espacamento AS "Espaçamento",
    dcr_material_genetico AS "Mat.Genético",
    dcr_especie AS "Espécie",
    data_plantio AS "Plantio",
    dcr_regime AS "Regime",
    vlr_area AS "Área(ha)",
    vlr_area_gis AS "Área GIS",
    tip_reg,
    data_reg,
    data_import_date,
    dcr_extra_02  AS "Tipo de Contrato",
    data_extra_02 AS "Início vigência", 
    data_extra_03 AS "Fim vigência",
    dcr_extra_05 as "Localidade" ,
    dcr_extra_06 as "Regional silvicultura",
    dcr_extra_07 as "Região climática"
   
FROM
    iforestry.vw_cubo_uso_solo
WHERE
    dcr_regiao = 'BRACELL SP'

    
"""

)

dados_cadastro  = cursor_cadastro.fetchall()

cursor_cadastro.close()


colunas_cadastro = [desc[0] for desc in cursor_cadastro.description]

cadastro = pd.DataFrame(dados_cadastro, columns=colunas_cadastro)


In [16]:
#Casting de área cadastro

cadastro['Área(ha)'] = pd.to_numeric(cadastro['Área(ha)'])

cadastro['Área GIS'] = pd.to_numeric(cadastro['Área GIS'])

In [17]:
#Cadastro


#Inputando premissas de cadastro
premissas_cadastro = pd.read_excel(path_premissas, sheet_name='status_cadastro', engine='calamine')

# #Definindo caminho do cadastro
# path_cadastro = input_file('path_cadastro')

# #Ler excel
# cadastro = pd.read_excel(path_cadastro, dtype={'Talhão':'object', 'Id Projeto':'object', 'Projeto e Talhão':'object'})



#Filtro Região
cadastro = cadastro[cadastro['Região'] == 'BRACELL SP']

#Status Descrição
cadastro = cadastro.merge(premissas_cadastro[['Descrição do Uso do talhão','Status_descrição']], on='Descrição do Uso do talhão', how='left')

#Status contrato
cadastro = cadastro.merge(premissas_cadastro[['Tipo de Contrato','Status_contrato']], on='Tipo de Contrato', how='left')


cadastro = cadastro.rename({'Projeto e Talhão':'Objeto de locação'},axis=1)


In [18]:
#Cadastro & Coppice

cadastro = cadastro.merge(coppice[['Objeto de locação','IMA', 'coppice','ha coppice']], on='Objeto de locação', how='left')



In [19]:
#Cadastro & Rehab

cadastro = cadastro.merge(rehab, on='Objeto de locação', how='left')

cadastro['Área(ha) Cadastro'] = cadastro['Área(ha)']

cadastro['Área(ha)'] = np.where(pd.notna(cadastro['Rehab status']),cadastro['area rehab'], cadastro['Área(ha)'])



#Retirando tudo o que já foi plantado
data_rehab = pd.Timestamp(day=1, month=10, year=2024)

cadastro = cadastro[~ ((pd.notna(cadastro['Rehab status'])) & (cadastro['Plantio'] >= data_rehab))]


In [20]:
#Vento

path_vento = input_file('path_vento')


vento = pd.read_excel(path_vento,
                      sheet_name='BD WD',
                      dtype={'Id Fazenda':'object',
                             'Talhão':'object'},
                      usecols=['Id Fazenda','Talhão','Houve recomendação\r\n de reforma?','Área recomendada para reforma'],
                      engine='calamine').rename({'Houve recomendação\r\n de reforma?': 'Recomendação Reforma',
                                                 'Área recomendada para reforma': 'area vento'},axis=1)

vento = vento[((vento['Recomendação Reforma'] != 'Não') & (pd.notna(vento['Recomendação Reforma'])) & (pd.notna(vento['area vento'])))]


vento['vento'] = 'vento'

vento['Objeto de locação'] = vento['Id Fazenda'].astype(str) + vento['Talhão'].astype(str)

vento = vento.drop(['Id Fazenda', 'Talhão'], axis=1)

ajuste_obj(vento,'Objeto de locação')

In [21]:
#Cadastro & Vento

cadastro = cadastro.merge(vento, on='Objeto de locação', how='left')


cadastro['Área(ha)'] = np.where(pd.notna(cadastro['vento']), cadastro['area vento'], cadastro['Área(ha)'])

In [22]:
#Cadastro & Espaçamento P&D

cadastro = cadastro.merge(esp, on= 'Objeto de locação', how='left')

In [23]:
#Cadastro Filtros

cadastro = cadastro[(cadastro['Status_contrato'] == 'ativo') | 
                    (cadastro['Id Projeto'] == '1080' )]


#Se os campos forem vazios ele não aplicará o filtro de forma correta
cadastro = cadastro.fillna({'Status_descrição': '', 'coppice': '', 'Rehab status': '', 'vento': ''})

cadastro = cadastro[(cadastro['Status_descrição'] == 'ativo') |
                    (cadastro['coppice'] == 'coppice') | 
                    (cadastro['Rehab status'] == 'Rehab') |
                    (cadastro['vento'] == 'vento')]




#Escolhendo colunas de interesse
cadastro = cadastro[['Região','Id Projeto','Projeto','Talhão','Localidade','Regional silvicultura','Região climática',
                     'Ciclo','Regime','Uso do Talhão', 'Descrição do Uso do talhão','Plantio',
                     'Área(ha)','Tipo de Contrato','Início vigência','Objeto de locação', 
                     'Status_descrição', 'Status_contrato','IMA', 'coppice','Rehab',
                     'Rehab status','Área(ha) Cadastro','Recomendação Reforma', 'area vento', 'vento', 'Espaçamento P&D']]


In [24]:
#CZs prime

cz = pd.read_excel(path_premissas, sheet_name='cz', dtype={'cz': 'object'}, engine='calamine')

cz = cz[pd.notna(cz['cz'])]

In [25]:
#CZ & Cadastro

cadastro = cadastro.merge(cz, on='Localidade', how='left')

cadastro['Região climática'] = np.where( pd.notna(cadastro['cz']),
                     cadastro['Região climática'] + " - " + cadastro['cz'].astype(str),
                     cadastro['Região climática'])



In [26]:
#Capacidade de Plantio

path_capacidade = input_file('path_capacidade')

capacidade_plantio = pd.read_excel(path_capacidade,sheet_name='cap plantio')


capacidade_plantio = capacidade_plantio.melt(id_vars=['Service Provider'], 
                                    value_vars=[ '2025_01', '2025_02',
                                                '2025_03', '2025_04', '2025_05', '2025_06', '2025_07', '2025_08',
                                                '2025_09', '2025_10', '2025_11', '2025_12'],
                                    var_name='[m] Mês Operacional',
                                    value_name='Soma de [c] Valor de Produção',
                                    )


#Renomear coluna
capacidade_plantio = capacidade_plantio.rename({'Service Provider': 'Equipe de Plantio'}, axis=1)

#Alterando o tipo de dado númerico
capacidade_plantio['Soma de [c] Valor de Produção'] = pd.to_numeric(capacidade_plantio['Soma de [c] Valor de Produção'], errors = 'coerce')

capacidade_plantio['[m] Mês Operacional'] = pd.to_datetime(capacidade_plantio['[m] Mês Operacional'],format='%Y_%m')

In [27]:
#Micro

path_micro = input_file('path_micro')

micro = pd.read_excel(path_micro, 
                      sheet_name='MATRIZ',
                      skiprows=1,
                      usecols=['TALHÃO (PARA)','ID','Savannah','Rebaixador','Limpa Trilho'], dtype={'TALHÃO (PARA)':'object','ID':'object'},
                      engine = 'calamine')

#Objeto de locação
micro['Objeto de locação'] = micro['ID'].astype(str) + micro['TALHÃO (PARA)'].astype(str)

#Removendo duplicatas
micro = micro.drop_duplicates('Objeto de locação', keep='first')

#Ajustando a base do micro para talhão adequado

ajuste_obj(df=micro, colums='Objeto de locação')


In [28]:
#Proximidade
path_proximidade = input_file('path_proximidade')

proximidade = pd.read_excel(path_proximidade, 
                            sheet_name='dist', 
                            usecols=['FAZ', 'CIDADE', 'Total_Kilometers'],
                            dtype={'FAZ':'object'},
                            engine = 'calamine')

p_desc = ['RECHÃ','n 1 - GN', 'n 1 - INOVESA',
       'n 1 - CARPELO', 'n 1 - GERS', 'n 1 - EMFLORA', 'n 1 - GERAÇÃO',
       'n 1 - JFI DUARTINA', 'n 1 -  GN PRESIDENTE ALVES',
       'n 1 - PROPRIO', 'n 1 - SOLLUM', 'n 1 - JFI ITATINGA',
       'n 1 - NILZA', 'n 1 - JFI DOURADO', 'n 1 - RECHÃ',
       'n 1 - JFI CAPÃO BONITO', 'n 1 - MIGRAR',' GN PRESIDENTE ALVES'] 

proximidade = proximidade[~ proximidade['CIDADE'].isin(p_desc)]


proximidade = mascara(proximidade, 'FAZ', 4)


p_masc = {'SOLLUM':'SOLLUM',
          'NILZA':'NILZA',
          'PROPRIO':'BRACELL',
          'GERAÇÃO':'GERAÇÃO',
          'JFI DUARTINA':'JFI DUARTINA',
          'EMFLORA':'EMFLORA',
          'JFI ITATINGA':'JFI ITATINGA',
          'JFI DOURADO':'JFI DOURADO',
          'CARPELO':'CARPELO',
          'GERS':'GERS',
          'INOVESA':'INOVESA',
          'MIGRAR':'MIGRAR',
          'GN':'GN',
          'JFI CAPÃO BONITO':'JFI CAPÃO BONITO',
          'RA':'RA'
          }

proximidade = proximidade.sort_values(['FAZ','Total_Kilometers'], ascending=[True,True])

proximidade['EPS'] = proximidade['CIDADE'].map(p_masc)


proximidade['Sequencia'] ='EPS - ' + (proximidade.groupby(['FAZ']).cumcount()+1).astype(str)


#Proximidade Equipe
proximidade_eps = proximidade.pivot(index='FAZ', columns='Sequencia', values='EPS').reset_index().rename({'FAZ':'Id Projeto'},axis=1)
#Removendo duplicatas
proximidade_eps = proximidade_eps.drop_duplicates('Id Projeto', keep='first')

#Proximidade KM
proximidade_km = proximidade.pivot(index='FAZ', columns='Sequencia', values='Total_Kilometers').reset_index().rename({'FAZ':'Id Projeto'},axis=1)
#Removendo duplicatas
proximidade_km = proximidade_km.drop_duplicates('Id Projeto', keep='first')



In [29]:
#Cadastro & Micro

cadastro = cadastro.merge(micro[['Savannah', 'Rebaixador', 'Limpa Trilho', 'Objeto de locação']], on='Objeto de locação', how='left')


In [30]:
#Liberação de Áreas

path_lib = input_file('path_lib_area')

lib_area = pd.read_excel(path_lib,
                         sheet_name='base_liberação',
                         skiprows= 1,
                         usecols=['Id Projeto','Talhão','Status final - com ajuste de áreas < 40 ha','Bloqueado (SIM ou NÂO)','Motivo','Status Atual','Regional Plantio'],
                         dtype={'Id Projeto':'object','Talhão':'object'},
                         engine='calamine')

lib_area['Objeto de locação'] = lib_area['Id Projeto'].astype(str) + lib_area['Talhão'].astype(str)

lib_area = lib_area.rename({'Bloqueado (SIM ou NÂO)':'Bloqueio Lib', 
                            'Motivo': 'Motivo Lib',
                            'Status final - com ajuste de áreas < 40 ha': 'Status Lib', 
                            'Status Atual':'Status Atual - Lib',
                            'Regional Plantio':'Regional Plantio - Lib'},axis=1)

lib_area = lib_area.drop_duplicates('Objeto de locação')


#Ajustando a base de liberação de áreas para talhão adequado

ajuste_obj(df=lib_area, colums='Objeto de locação')


In [31]:
#Cadastro & Lib Área

cadastro = cadastro.merge(lib_area[['Objeto de locação','Bloqueio Lib','Motivo Lib','Status Lib','Status Atual - Lib','Regional Plantio - Lib']], on='Objeto de locação', how='left')

In [32]:
#Data Start - Área Bloqueada

path_area_bloqueada = path_premissas

area_bloqueada = pd.read_excel(path_area_bloqueada,sheet_name='area_bloqueada',
                               dtype={'Data Start - Fora RF': 'datetime64[ns]'})


area_bloqueada = area_bloqueada.rename({'Status':'Status Atual - Lib'}, axis=1)

In [33]:
#Cadastro & área bloqueada

cadastro = cadastro.merge(area_bloqueada, on='Status Atual - Lib', how='left')

In [34]:
#PAC

path_pac = input_file('path_pac')

pac = pd.read_excel(path_pac, 
                        sheet_name='DB_SEQUÊNCIA',
                        skiprows=24,
                        dtype={'ID Faz.':'object', 'Talhão':'object'},
                        usecols=['ID Faz.','Talhão','Mês Corte','Área','Tipo Contrato'], engine='calamine')
    

#pac['Mês Corte'] = pd.to_numeric(pac['Mês Corte'], errors='coerce')
#pac['Mês Corte'] = pd.to_datetime(pac['Mês Corte'], unit='D', origin='1899-12-30')

pac = pac[(pd.notna(pac['Mês Corte'])) & (pac['Mês Corte'] != '-')]

pac = pac.rename({'Talhão':'Objeto de locação'},axis=1) 
pac = pac.drop_duplicates('Objeto de locação', keep='first')


#Ajustando a base do pac para talhão adequado

ajuste_obj(df=pac, colums='Objeto de locação')

#pac.to_excel('pac.xlsx')

In [35]:
#PAB

path_pab = input_file('path_pab')

pab = pd.read_excel(path_pab, 
                        sheet_name='BS_Sequencia',
                        skiprows=20,
                        dtype={'ID Faz.':'object', 'Talhão':'object'},
                        usecols=['ID Faz.','Talhão','Mês Início'], engine='calamine')
    

# pab['Mês Início'] = pd.to_numeric(pab['Mês Início'], errors='coerce')
# pab['Mês Início'] = pd.to_datetime(pab['Mês Início'], unit='D', origin='1899-12-30')

pab = pab[(pd.notna(pab['Mês Início'])) & (pab['Mês Início'] != '-')]

pab = pab.rename({'Talhão':'Objeto de locação','Mês Início':'Mês Baldeio'},axis=1) 
pab = pab.drop_duplicates('Objeto de locação', keep='first')


#Ajustando a base do pab para talhão adequado
ajuste_obj(df=pab, colums='Objeto de locação')

In [36]:
#PAS

path_pas = input_file('path_pas')

pas = pd.read_excel(path_pas, 
                        sheet_name='BD',
                        skiprows=1,
                        dtype={'Id Projeto':'object', 'Talhão':'object'},
                        usecols=['Id Projeto','Talhão','Data liberação ajustada','Bloqueio operacional'], 
                        engine="calamine")


pas = pas[pas['Bloqueio operacional'] == 0]
pas['Objeto de locação'] = pas['Id Projeto'].astype(str) + pas['Talhão'].astype(str)

pas = pas[['Id Projeto','Data liberação ajustada', 'Objeto de locação']]

pas = pas.drop_duplicates('Objeto de locação', keep='first')


#Ajustando a base do pas para talhão adequado
ajuste_obj(df=pas, colums='Objeto de locação')

In [37]:
#RAF

path_raf = input_file('path_raf')

raf = pd.read_excel(path_raf,
                    sheet_name='Sheet1',
                    usecols=['Id Projeto','Talhão','Base.Tipo de Ocorrência'],
                    dtype={'Id Projeto':'object', 'Talhão':'object'}, 
                    engine='calamine'
                    )

raf = raf.rename({'Base.Tipo de Ocorrência': 'Raf'},axis=1)

raf['Objeto de locação'] = raf['Id Projeto'].astype(str) + raf['Talhão'].astype(str)

raf = raf.drop_duplicates('Objeto de locação', keep='first')

#Ajustando a base do raf para talhão adequado
ajuste_obj(df=raf, colums='Objeto de locação')

raf = raf[pd.notna(raf['Raf'])]

In [38]:
#Cadastro; Raf; PAS; PAB; PAC

#Cadastro e RAF
cadastro = cadastro.merge(raf[['Objeto de locação', 'Raf']], on= 'Objeto de locação', how='left')

#Cadastro e PAS
cadastro = cadastro.merge(pas[['Data liberação ajustada','Objeto de locação']], on='Objeto de locação', how='left')

#Cadastro e Pab
cadastro = cadastro.merge(pab[['Objeto de locação', 'Mês Baldeio']],on='Objeto de locação', how='left')

#Cadastro e Pac
cadastro = cadastro.merge(pac[['Objeto de locação','Mês Corte']], on='Objeto de locação', how='left')

In [39]:
#Fazendas por Regional

path_faz_reg = input_file("path_fazenda_regional")

faz_reg = pd.read_excel(path_faz_reg,
                        sheet_name='BASE FAZENDAS TALHÕES TECNICOS',
                        skiprows=1,
                        usecols=['Id Projeto','Talhão','COORDENAÇÃO DE PLANTIO', 'EPS DE PLANTIO', 'COORDENAÇÃO DO BLOCO', 'SUPERVISOR DO BLOCO'],
                        dtype={'Id Projeto':'object', 'Talhão':'object'},
                        engine='calamine')

faz_reg['Objeto de locação'] = faz_reg['Id Projeto'].astype(str) + faz_reg['Talhão'].astype(str)

faz_reg = faz_reg.rename({'COORDENAÇÃO DE PLANTIO': 'Coordenação Plantio',
                          'EPS DE PLANTIO':'EPS Plantio',
                          'COORDENAÇÃO DO BLOCO': 'Coordenação Bloco',
                          'SUPERVISOR DO BLOCO':'Supervisor Bloco'}, axis=1)

faz_reg = faz_reg.drop_duplicates('Objeto de locação',keep='first')


#Ajustando a base fazendas por regional para talhão adequado
ajuste_obj(df=faz_reg, colums='Objeto de locação')


In [40]:
#Cadastro e Fazendas por Regional

cadastro = cadastro.merge(faz_reg[['Objeto de locação', 'Coordenação Bloco', 'Coordenação Plantio','EPS Plantio', 'Supervisor Bloco']], on='Objeto de locação', how='left')

cadastro['Coordenação Bloco'] = np.where(pd.notna(cadastro['Regional silvicultura']), 
                                         cadastro['Regional silvicultura'],
                                         cadastro['Coordenação Bloco'])


cadastro['Group'] = np.where( cadastro['coppice'] == 'coppice', cadastro['coppice'],
                                np.where(
                                    cadastro['Rehab status'] == 'Rehab', cadastro['Rehab'],
                                
                                    np.where( cadastro['vento'] == 'vento', cadastro['vento'],
                                            cadastro['Regime'])
                                        )
                                )
                        

In [41]:
#RF

# path_rf = input_file('path_rf')

# rf = pd.read_excel(path_rf,
#                    sheet_name='BD',
#                    usecols=['Id Projeto', 'Talhão','RF PLANTIO','EPS','Regional Execução1'],
#                    dtype={'Talhão':'object', 'Id Projeto':'object'},
#                    engine='calamine')

# #Filtrando somente RF01
# rf = rf[rf['RF PLANTIO'] == 'RF01-25']

# rf['Objeto de locação'] = rf['Id Projeto'].astype(str) + rf['Talhão'].astype(str)

# rf = rf.drop_duplicates('Objeto de locação',keep='first')

# rf = rf.rename({'RF PLANTIO':'RF Plantio', 'Regional Execução1': 'Regional Execução', 'EPS': 'EPS Execução'},axis=1)


In [42]:
#Cadastro & RF

#cadastro = cadastro.merge(rf[['Objeto de locação','RF Plantio','Regional Execução', 'EPS Execução']], on='Objeto de locação', how='left')

In [43]:
#Cadastro & CTO

cadastro = cadastro.merge(cto[['cto baldeio', 'cto colheita','Objeto de locação']], on='Objeto de locação', how='left')

In [44]:
#Cadastro & Ano de plantio

today = datetime.today()

#Data de Plantio
cadastro['Plantio'] = pd.to_datetime(cadastro['Plantio'], dayfirst=True, errors='coerce')

#removendo data de plantio futura
cadastro['Plantio'] = np.where(
    (cadastro['Plantio'] > today) | 
    ((cadastro['Descrição do Uso do talhão'] == '102-Aguardando plantio') & (pd.notna(cadastro['Plantio']))), 
    None, 
    cadastro['Plantio']
)
#Transformando para data novamente
cadastro['Plantio'] = pd.to_datetime(cadastro['Plantio'], dayfirst=True, errors='coerce')

#Ano de Plantio
cadastro['Ano de Plantio'] = cadastro['Plantio'].dt.year


In [45]:
#Cadastro & Ano de Corte

cadastro['Mês Corte'] = pd.to_datetime(cadastro['Mês Corte'],errors='coerce', dayfirst=True)

cadastro['Ano de Corte'] = cadastro['Mês Corte'].dt.year

In [46]:
#Áreas sem expectativa de corte

#Desconsiderar o que não tem pac e pab e tem status crescimento
#CTO dos últimos dois meses considerar 


# cadastro['Sem previsão de corte'] = np.where((cadastro['RF Plantio'] == 'RF01-25'), 'Não',
#                                         np.where((pd.isna(cadastro['Mês Corte'])) & 
#                                             (pd.isna(cadastro['Mês Baldeio'])) &
#                                             (cadastro['Descrição do Uso do talhão'] == "104-Crescimento")
#                                             ,
#                                             'Sim',
#                                             'Não'
#                                             ))



cadastro['Sem previsão de corte'] = np.where(((pd.isna(cadastro['Mês Corte'])) | (pd.isna(cadastro['Mês Baldeio']))) &
                                            ((cadastro['Descrição do Uso do talhão'] == "108-Plantio comercial") | 
                                             (cadastro['Descrição do Uso do talhão'] == "203-Experimento restrito") |
                                             (cadastro['Descrição do Uso do talhão'] == "201-Experimento aguardando plantio")
                                              )
                                            ,
                                            'Sim',
                                            'Não'
                                           )



#Se for vazio considerar como não (Ele já considera automaticamente)

# cadastro['Plantio após PAC'] = np.where((cadastro['RF Plantio'] == 'RF01-25'), 'Não',
#                                         np.where(((cadastro['Ano de Plantio'] >=2022) & (cadastro['Ano de Plantio'] <=2024 ) & (pd.isna(cadastro['Raf']))) |
#                                             (cadastro['Plantio'] > cadastro['Mês Corte']) ,
#                                             'Sim',
#                                             'Não'))


cadastro['Plantio após PAC'] = np.where(((cadastro['Ano de Plantio'] >=2022) & (cadastro['Ano de Plantio'] <=2025 ) & (pd.isna(cadastro['Raf']))) |
                                            (cadastro['Plantio'] > cadastro['Mês Corte']),
                                            'Sim',
                                            'Não')





#Exporta um cadastro sem filtros para status de liberações
#cadastro.to_excel('cadastro_sem_filtro.xlsx')

#Retirando o que não tem previsão de corte e mantendo o rehab e vento


# cadastro = cadastro[(cadastro['Sem previsão de corte'] == 'Não') | 
#                     (cadastro['Rehab status'] == 'Rehab')|
#                     (cadastro['vento'] == 'vento')]

# #Retirando plantio de 2022 até 2025 e tudo que não tem expectativa de plantio e mantendo o rehab e vento

# cadastro = cadastro[(cadastro['Plantio após PAC'] == 'Não') | 
#                     (cadastro['Rehab status'] == 'Rehab')|
#                     (cadastro['vento'] == 'vento')]

In [47]:
#Remover Áreas Bloqueadas

cadastro = cadastro[(cadastro['Status Atual - Lib'] != 'Plantado - Aguardando SGF') | 
                    (cadastro['Rehab status'] == 'Rehab') |
                    (cadastro['vento'] == 'vento')
                    
                    ]

In [48]:
#Cadastro & Proximidade

cadastro = cadastro.merge(proximidade_eps[['Id Projeto','EPS - 1','EPS - 2','EPS - 3','EPS - 4']], on='Id Projeto',how='left')

cadastro = cadastro.merge(proximidade_km[['Id Projeto','EPS - 1','EPS - 2','EPS - 3','EPS - 4']],on='Id Projeto',how='left', suffixes=('',' km'))

In [49]:
#Data Start para os volumes de 2024

cadastro['Data Start'] =pd.to_datetime(
        cadastro['Data Start - Fora RF'] + pd.to_timedelta(cadastro['Delta Para Plantio'], unit='D'),
        pd.Timestamp(day=1, month=12, year=2024)
    )

In [50]:
#CTO & PAC & PAB

#CTO baldeio ajustado
cadastro['cto baldeio'] = np.where(cadastro['Mês Corte'] > cadastro['cto baldeio'], pd.NaT, cadastro['cto baldeio'])
cadastro['cto baldeio'] = pd.to_datetime(cadastro['cto baldeio'], errors='coerce')

#CTO baldeio ajustado de acordo ao baldeio
cadastro['cto baldeio'] = np.where(cadastro['Mês Baldeio'] > cadastro['cto baldeio'], cadastro['Mês Baldeio'], cadastro['cto baldeio'])
cadastro['cto baldeio'] = pd.to_datetime(cadastro['cto baldeio'], errors='coerce')


#CTO colheita ajustado
cadastro['cto colheita'] = np.where(cadastro['Mês Corte'] > cadastro['cto colheita'], pd.NaT, cadastro['cto colheita'])
cadastro['cto colheita'] = pd.to_datetime(cadastro['cto colheita'], errors='coerce')

In [51]:
#Data de referência

choice_ref = [pd.notna(cadastro['Data Start']),
            pd.notna(cadastro['cto baldeio']),
            ((pd.isna(cadastro['cto baldeio'])) & pd.notna(cadastro['cto colheita'])),
            ((pd.notna(cadastro['Mês Corte'])) & pd.isna(cadastro['cto colheita']) & (pd.isna(cadastro['cto baldeio']))),
            (pd.notna(cadastro['Mês Baldeio'])) & ((pd.isna(cadastro['Mês Corte'])) & pd.isna(cadastro['cto colheita']) & (pd.isna(cadastro['cto baldeio']))),
            ((cadastro['Regime'] == 'Implantação') & (pd.isna(cadastro['Mês Corte'])) & (pd.isna(cadastro['Data liberação ajustada']))),
            ((pd.notna(cadastro['Data liberação ajustada'])))

]

result_ref_name = ['Liberação de áreas',
                   'cto baldeio',
                   'cto colheita', 
                   'pac', 
                   'pab',
                   'Data de contrato',
                   'pas']


result_ref = [cadastro['Data Start'],cadastro['cto baldeio'], cadastro['cto colheita'], cadastro['Mês Corte'], cadastro['Mês Baldeio'],cadastro['Início vigência'],cadastro['Data liberação ajustada']]

cadastro['Data de Referência'] = pd.to_datetime(np.select(choice_ref, result_ref, default=pd.NaT), errors='coerce')

cadastro['Origem referência'] = np.select(choice_ref,result_ref_name, default=pd.NaT)

In [52]:
#Manejo

cadastro['Manejo'] = np.where((cadastro['Savannah'] == 'Obrigatório') & (cadastro['Rebaixador'] == 'Obrigatório'), 'savannah/rebaixador',
                                np.where((cadastro['Rebaixador'] == 'Obrigatório'),'rebaixamento',
                                    np.where((cadastro['Savannah'] == 'Obrigatório'), 'savannah', pd.NaT
                                        )))

cadastro['Manejo'] = np.where(pd.notna(cadastro['Manejo']), cadastro['Manejo'], cadastro['Origem referência'])

In [53]:
#Manejos Premissas

path_manejo = path_premissas

manejo = pd.read_excel(path_manejo,
                       sheet_name='plan',
                       engine='calamine')


In [54]:
#Data Máxima e Mínima de expectativa de plantio

cadastro['Data de Referência Min'] = cadastro.groupby('Id Projeto')['Data de Referência'].transform('min')

cadastro['Data de Referência Max'] = cadastro.groupby('Id Projeto')['Data de Referência'].transform('max')


In [55]:
#Cadastro & Manejo

cadastro = cadastro.merge(manejo, on='Manejo', how='left')


#Definindo data de operação

#Onde for nulo, trazer a data máxima da fazenda

#cadastro['Data de Referência'] = np.where(pd.isna(cadastro['Data de Referência']), cadastro['Data de Referência Max'],cadastro['Data de Referência'])

#Não somar duas vezes o rebaixamento
 
cadastro['Intervalo plantio'] = np.where( ((cadastro['Status Atual - Lib'] == 'Aguardando Savannah') | (cadastro['Status Atual - Lib'] == 'Aguardando Rebaixamento') | (cadastro['Status Atual - Lib'] == 'Área Bloqueada' )),
                                          0, 
                                          cadastro['Intervalo plantio'])


#Não posso somar duas vezes a mesma operação se ela já foi somada e tem data start
cadastro['Expectativa de Plantio'] = cadastro['Data de Referência'] + pd.to_timedelta(cadastro['Intervalo plantio'], unit='D')


cadastro = mes_operacional(df=cadastro, coluna='Expectativa de Plantio', nome_coluna='Expectativa de Plantio Op')

In [56]:
#Meses Ativos

path_p_ativo = path_premissas

periodo_ativo = pd.read_excel(path_p_ativo,
                              sheet_name='periodo_ativo',
                              engine='calamine'
                        )

#Cadastro & Período Ativo

cadastro = cadastro.merge(periodo_ativo, on='Expectativa de Plantio Op', how='left')

#Filtrando tudo que está desativado

cadastro = cadastro[cadastro['Status Expectativa'] != 'desativado']

In [57]:
#Ajustes

#Definir o que está no RF -> Mantém

#cadastro['Expectativa de Plantio Op'] = np.where((cadastro['RF Plantio'] == 'RF01-25'), pd.Timestamp(day=1, month=1, year=2025), cadastro['Expectativa de Plantio Op'])

cadastro['Expectativa de Plantio Op'] = pd.to_datetime(cadastro['Expectativa de Plantio Op'], errors='coerce')


#Data de referência para 2024
data_ref = pd.Timestamp(day=31, month=12, year=2024)
#Se a expectativa for menor que 2025, deve ficar em branco
cadastro['Expectativa de Plantio Op'] = np.where( (cadastro['Expectativa de Plantio Op'] > data_ref), cadastro['Expectativa de Plantio Op'], pd.NaT)
cadastro['Expectativa de Plantio Op'] = pd.to_datetime(cadastro['Expectativa de Plantio Op'], errors='coerce')



In [58]:
#Pulando o que não está em RF

rf_01 = pd.Timestamp(day=1, month=1, year=2025)
rf_02 = pd.Timestamp(day=1, month=2, year=2025)

cadastro['Expectativa de Plantio Op'] = np.where(((cadastro['Expectativa de Plantio Op'] == rf_01) & (cadastro['Manejo'] != 'RF')),
                                                rf_02, cadastro['Expectativa de Plantio Op'])
                                                 

cadastro['Expectativa de Plantio Op'] = pd.to_datetime(cadastro['Expectativa de Plantio Op'], errors='coerce')

In [59]:
path_replanejamento = path_premissas

replanejamento = pd.read_excel(path_replanejamento,
                               sheet_name='replanejamento',
                               usecols=['Id Projeto','Talhão','Nova Coordenação Plantio','Nova EPS Plantio','Nova Expectativa de Plantio','Observação'],
                               dtype={'Id Projeto':'object','Talhão':'object'},
                               engine='calamine')

replanejamento['Objeto de locação'] = replanejamento['Id Projeto'].astype(str) + replanejamento['Talhão'].astype(str)

replanejamento = replanejamento.drop(['Id Projeto','Talhão'],axis=1)


#Ajustando talhão no replanejamento
ajuste_obj(df=replanejamento, colums='Objeto de locação')


In [60]:
#Output que geram as informações para replanejamento

output_faz = cadastro[['Id Projeto', 'Projeto', 'Talhão','Área(ha)','Descrição do Uso do talhão','Localidade','Data de Referência','Manejo','Group',
                       'Coordenação Bloco','Coordenação Plantio','Supervisor Bloco','EPS Plantio','Expectativa de Plantio Op','EPS - 1', 'EPS - 2','EPS - 3', 'EPS - 4', 
                       'EPS - 1 km', 'EPS - 2 km', 'EPS - 3 km',
       'EPS - 4 km']].copy()

output_faz['Objeto de locação'] = output_faz['Id Projeto'].astype(str) + output_faz['Talhão'].astype(str)


#Ajustando talhão no output_faz
ajuste_obj(df=output_faz, colums='Objeto de locação')


#Trazer o planejamento
output_faz = output_faz.merge(replanejamento, on='Objeto de locação', how='left')




#Resolvendo os filtros para fazer o planejamento de forma mais fácil


#_______________________________________________________________________________________________________#

#Alterando a EPS de Plantio para a atual
output_faz['EPS Plantio'] = np.where(pd.notna(output_faz['Nova EPS Plantio']),
                                     output_faz['Nova EPS Plantio'],
                                     output_faz['EPS Plantio']
                                     )

output_faz['Coordenação Plantio'] = np.where(pd.notna(output_faz['Nova Coordenação Plantio']),
                                     output_faz['Nova Coordenação Plantio'],
                                     output_faz['Coordenação Plantio']
                                     )

map_coord = {'Centro':'Centro',
             'CENTRO':'Centro',
             'Norte':'Norte',
            'NORTE':'Norte',
            'SUL':'Sul',
            'Sul':'Sul',
            'NOROESTE':'Noroeste',
            'BSP':'BSP',
            'Próprio':'BSP'}

output_faz['Coordenação Plantio'] = output_faz['Coordenação Plantio'].map(map_coord)



#Alterando a data de plantio para a atual 
output_faz['Data Plantio'] = output_faz['Nova Expectativa de Plantio'].astype(str).str[-5:]

output_faz['Data Plantio'] = pd.to_datetime(output_faz['Data Plantio'], format='%m-%y', errors='coerce')


output_faz['Data liberada Plantio'] = output_faz['Expectativa de Plantio Op']

output_faz['Expectativa de Plantio Op'] = np.where(pd.notna(output_faz['Data Plantio']),
                                                 output_faz['Data Plantio'],
                                                 output_faz['Expectativa de Plantio Op'])

output_faz['Expectativa de Plantio Op'] = pd.to_datetime(output_faz['Expectativa de Plantio Op'], errors='coerce')

output_faz = output_faz.drop('Data Plantio', axis=1)

#_______________________________________________________________________________________________________#

output_faz = output_faz.drop_duplicates('Objeto de locação', keep='first')

output_faz = output_faz.drop('Objeto de locação', axis=1)



output_faz = output_faz[['Id Projeto', 'Projeto', 'Talhão', 'Área(ha)',
       'Descrição do Uso do talhão', 'Localidade', 'Data de Referência','Manejo','Group','Coordenação Bloco',
       'Coordenação Plantio','Supervisor Bloco', 'EPS Plantio', 'Data liberada Plantio', 'Expectativa de Plantio Op',
       'EPS - 1', 'EPS - 2', 'EPS - 3', 'EPS - 4', 'EPS - 1 km', 'EPS - 2 km',
       'EPS - 3 km', 'EPS - 4 km', 'Nova Coordenação Plantio',
       'Nova EPS Plantio', 'Nova Expectativa de Plantio', 'Observação']]


output_faz.to_excel('output_fazendas.xlsx',index=False)

In [61]:
#Replanejamento

path_replan = path_premissas

replanejamento = pd.read_excel(path_replan,
                               sheet_name='replanejamento',
                               engine='calamine',
                               dtype={'Talhão':'object','Id Projeto':'object'})

#Objeto de locação
replanejamento['Objeto de locação'] = replanejamento['Id Projeto'].astype(str) + replanejamento['Talhão'].astype(str)

#Ajustando talhão no replanejamento
ajuste_obj(df=replanejamento, colums='Objeto de locação')

#Escolhendo colunas
replanejamento = replanejamento[['Objeto de locação','Nova Coordenação Plantio', 'Nova EPS Plantio','Nova Expectativa de Plantio','Observação']]


#Levar para o cadastro as novas considerações

replanejamento['Nova Expectativa de Plantio'] = replanejamento['Nova Expectativa de Plantio'].astype(str).str[-5:]

replanejamento['Nova Expectativa de Plantio'] = pd.to_datetime(replanejamento['Nova Expectativa de Plantio'], format='%m-%y', errors='coerce')


#Removendo as linhas onde temos todas as informações vazias
replanejamento = replanejamento[((pd.notna(replanejamento['Nova Coordenação Plantio'])) | 
                                 (pd.notna(replanejamento['Nova EPS Plantio'])) | 
                                 (pd.notna(replanejamento['Nova Expectativa de Plantio'])) |
                                 (pd.notna(replanejamento['Observação']))
                                 )]

In [62]:
#Cadastro & Replanejamento

cadastro = cadastro.merge(replanejamento, on='Objeto de locação', how='left')


#Alterando coordenação de plantio
cadastro['Coordenação Plantio'] = np.where(pd.notna(cadastro['Nova Coordenação Plantio']),
                                         cadastro['Nova Coordenação Plantio'],
                                         cadastro['Coordenação Plantio'])

map_coord = {'Centro':'Centro',
             'CENTRO':'Centro',
             'Norte':'Norte',
            'NORTE':'Norte',
            'SUL':'Sul',
            'Sul':'Sul',
            'NOROESTE':'Noroeste',
            'BSP':'BSP',
            'Próprio':'BSP'}

cadastro['Coordenação Plantio'] = cadastro['Coordenação Plantio'].map(map_coord)


#Alterando equipe de plantio
cadastro['EPS Plantio'] = np.where(pd.notna(cadastro['Nova EPS Plantio']),
                                         cadastro['Nova EPS Plantio'],
                                         cadastro['EPS Plantio'])


cadastro['Data liberada Plantio'] = cadastro['Expectativa de Plantio Op']

#Alterando expectativa de plantio
cadastro['Expectativa de Plantio Op'] = np.where(pd.notna(cadastro['Nova Expectativa de Plantio']),
                                         cadastro['Nova Expectativa de Plantio'],
                                         cadastro['Expectativa de Plantio Op'])

cadastro['Expectativa de Plantio Op'] = pd.to_datetime(cadastro['Expectativa de Plantio Op'], errors='coerce') 

In [63]:
#Cadastro & Proximidade
cadastro = cadastro.merge(proximidade[['FAZ','EPS','Total_Kilometers']], left_on=['Id Projeto','EPS Plantio'], right_on=['FAZ','EPS'],how='left').drop(['FAZ', 'EPS'],axis=1).rename({'Total_Kilometers':'Distância km'})



In [64]:
#Removendo duplicatas de cadastro
cadastro = cadastro.drop_duplicates('Objeto de locação', keep='first')

In [65]:
#Volume Planejado



cadastro['Área(ha)'] = cadastro['Área(ha)'].astype(float)
cadastro['Total_Kilometers'] = cadastro['Total_Kilometers'].astype(float)

cap_agg= {'Área(ha)':'sum',
          'Total_Kilometers':'mean'}

capacidade_real = cadastro.groupby(['EPS Plantio','Expectativa de Plantio Op']).agg(cap_agg).reset_index()



In [66]:
#Capacidade real

capacidade_real = capacidade_real.merge(capacidade_plantio, left_on =['EPS Plantio','Expectativa de Plantio Op'], 
                                        right_on =['Equipe de Plantio', '[m] Mês Operacional'], 
                                        how='left').drop(['Equipe de Plantio','[m] Mês Operacional'],axis=1).rename({'Soma de [c] Valor de Produção':'Capacidade','Total_Kilometers': 'Km Médio'}, axis=1)

capacidade_real	= capacidade_real[['EPS Plantio', 'Expectativa de Plantio Op', 'Área(ha)','Capacidade','Km Médio']]

In [67]:
coordenacao_dict = {
    'SOLLUM': 'CENTRO',
    'MIGRAR': 'SUL',
    'EMFLORA': 'NORTE',
    'GERAÇÃO': 'NOROESTE',
    'GN': 'NORTE',
    'NILZA': 'CENTRO',
    'GERS': 'NORTE',
    'INOVESA': 'NOROESTE',
    'CARPELO': 'NOROESTE',
    'RA FLORESTAL': 'SUL',
    'JFI DUARTINA': 'NORTE',
    'JFI DOURADO': 'SUL',
    'JFI CAPÃO BONITO': 'SUL',
    'JFI ITATINGA': 'SUL',
    'BRACELL 01': 'BSP' ,
    'BRACELL 02': 'BSP'
}

capacidade_real['Regional'] = capacidade_real['EPS Plantio'].map(coordenacao_dict)

In [68]:
#Análise das fazendas
cadastro['area total'] = cadastro.groupby('Id Projeto')['Área(ha)'].transform('sum')

cadastro['Projetos < 40'] = np.where(cadastro['area total'] <= 40 , 'Projetos < 40', 'Projetos > 40')


In [69]:
#EPS Ideal dentro da regional


eps_regional = pd.read_excel(path_premissas,sheet_name='eps', usecols=['eps_para','regional']).drop_duplicates('eps_para')


map_esp_regional = {'CE':'Centro',
                    'NO': 'Norte',
                    'NW':'Noroeste',
                    'SU': 'Sul'}


eps_regional['regional'] = eps_regional['regional'].map(map_esp_regional)


eps_regional = eps_regional[pd.notna(eps_regional['regional'])]




eps_regional['eps_para'] = eps_regional['eps_para'].str.upper().replace({'BRACELL 1': 'BRACELL'})




#Defindo melhor eps por cadastro

cadastro_eps = cadastro.groupby(['Id Projeto','Projeto','Coordenação Bloco', 'EPS - 1','EPS - 2', 'EPS - 3', 'EPS - 4', 
                                 'EPS - 1 km', 'EPS - 2 km','EPS - 3 km', 'EPS - 4 km'])['Área(ha)'].sum().reset_index()


cadastro_eps = cadastro_eps.sort_values('Área(ha)', ascending=False)

cadastro_eps = cadastro_eps.drop_duplicates('Id Projeto', keep='first').drop('Área(ha)', axis=1)



#Função para encontrar qual é a melhor regional

def mapear_eps(row, eps_regional):
    # Filtra os EPS disponíveis para a regional correspondente
    eps_disponiveis = eps_regional.loc[eps_regional['regional'] == row['Coordenação Bloco'], 'eps_para'].tolist()
    
    # Testa cada EPS (de 1 a 4) para ver se está na lista de eps_disponíveis
    for col in ['EPS - 1', 'EPS - 2', 'EPS - 3', 'EPS - 4']:
        if row[col] in eps_disponiveis:
            return row[col]  # Retorna o primeiro EPS encontrado na lista
    
    # Se nenhum EPS foi encontrado, retorna o primeiro disponível na lista
    return eps_disponiveis[0] if eps_disponiveis else None



cadastro_eps['EPS Ideal Blocagem'] = cadastro_eps.apply(lambda row: mapear_eps(row, eps_regional), axis=1)

In [70]:
cadastro_eps.to_excel('eps_fazenda.xlsx')

In [71]:
#Fazendas sem RF

fz_s_rf = cadastro[pd.isna(cadastro['Expectativa de Plantio Op'])][['Projeto']].drop_duplicates('Projeto')



In [72]:
#Oportunidade de antecipar data de plantio


fz_ant = cadastro[pd.notna(cadastro['Expectativa de Plantio Op'])]

fz_ant = fz_ant[fz_ant['Projetos < 40'] != 'Projetos < 40']

#Aparecendo poucos talhões por fazenda, encontrar um percentual e ver qual deve ser considerado, se o número de talhões for baixo -> exemplo da recreio com talhões de antigos pequenos

fz_ant['% rep'] = round((fz_ant['Área(ha)']  / fz_ant.groupby('Projeto')['Área(ha)'].transform('sum')) * 100,2)

#Somar a representatividade que está no mesmo mês de expectativa de plantio 

fz_ant['% rep'] = fz_ant.groupby(['Expectativa de Plantio Op','Projeto'])['% rep'].transform('sum')


fz_ant =  fz_ant[fz_ant['Expectativa de Plantio Op'] > fz_ant['Data liberada Plantio']]

fz_ant = fz_ant[fz_ant['% rep'] > 70][['Projeto']].drop_duplicates('Projeto')



In [73]:
#Capacidade Export
capacidade_real.to_excel('capacidade.xlsx', index=False)

In [74]:
#Distribuição Export
cadastro.to_excel("Distruibuição Regional.xlsx", index=False)

In [75]:
#Sugestão de mudança

#Gerar Export para sugerir fazendas que podem ser melhores opções para outras regionais

#Premissa

#Olhar a capacidade

#Olhar para pontos de atenção, que são as eps e quanto elas estão acima da capidade -> > 30% da capacidade, sugestão de alterar dono da fazenda

#Equipe mais próxima (Se Não tiver capacidade, qual é a próxima) - Como definir qual fazenda deve ser desconsiderada -> A mais longe

#Projeto Sugestão de mudança


In [76]:
#Projetos
path_mapa = input_file('path_mapas')

projetos = gd.read_file(path_mapa, drive = 'GeoJSON')

projetos = projetos.drop(['DATA_REG','AREA_HA','CD_UNIDADE'],axis=1)

projetos = projetos[projetos['CD_REGIAO'] == 1 ]

#EPS
projetos = projetos.to_crs(epsg = 31983)

projetos = projetos.set_geometry('geometry')

c:\Users\gamoura\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver ESRI Shapefile does not support open option DRIVE
  return ogr_read(


In [77]:
#RF

projetos = projetos.merge(cadastro[['Id Projeto','EPS Plantio','Data liberada Plantio','Expectativa de Plantio Op','Coordenação Plantio']].sort_values(['Id Projeto','Expectativa de Plantio Op'],ascending=[True,True]).drop_duplicates('Id Projeto',keep='first'),
                           left_on='ID_PROJETO',
                             right_on='Id Projeto')


#RF
projetos['RF'] = 'RF - ' + projetos['Expectativa de Plantio Op'].dt.month.fillna(0).astype(int).astype(str).str.zfill(2)

#projetos = projetos.drop('Expectativa de Plantio Op',axis=1)


In [78]:
#Save shape

save_shp = r"F:\Silvicultura\01. SP\PROCESSOS\COI Silvicultura - Edicao\01. Programações & Controles\001. Rolling Forcast - RF\Distribuição Regional\output mapas\distribuicao_operacional.shp"


projetos.to_file(save_shp,
                  driver="ESRI Shapefile",
                  engine='pyogrio')

C:\Users\gamoura\AppData\Local\Temp\ipykernel_32144\2615104559.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  projetos.to_file(save_shp,
c:\Users\gamoura\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'EPS Plantio' to 'EPS Planti'
  ogr_write(
c:\Users\gamoura\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Data liberada Plantio' to 'Data liber'
  ogr_write(
c:\Users\gamoura\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Field Data liber create as date field, though DateTime requested.
  ogr_write(
c:\Users\gamoura\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Expectativa de Plantio Op' to 'Expectativ'
  ogr_write(
c:\Users\gamoura\AppData\Lo